In [1]:
#Importing Librabries


import cv2 
import numpy as np
import os 
import matplotlib.pyplot as plt
import time 
import mediapipe as mp


In [2]:



mp_holistic=  mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable =False 
    results = model.process(image)
    image.flags.writeable =True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
##rendering Landmarks 
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    

In [5]:
def draw_landmarks2(image,result):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius =1 ),
                              mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius =1 )
                              
                             
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius =1 ),
                              mp_drawing.DrawingSpec(color=(80,44,10),thickness=2,circle_radius =1 )
                              )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius =1 ),
                              mp_drawing.DrawingSpec(color=(80,44,10),thickness=2,circle_radius =1 )
                              )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,117,10),thickness=2,circle_radius =1 ),
                              mp_drawing.DrawingSpec(color=(80,66,10),thickness=2,circle_radius =1 )
                              )
    
    

In [6]:

##Extracting Landmarks 
"""pose =[]
for res in results.pose_landmarks.landmark:
    test = np.array([res.x,res.y,res.z,res.visibility])
    pose.append(test)"""

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

test = extract_keypoints(results)
test.shape

NameError: name 'results' is not defined

In [6]:
##Tracking Face and Hands
i = 0
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        #Using Mediapipe

        image, results = mediapipe_detection(frame, holistic)   
        
        #Drawing Ladmarks
        draw_landmarks2(image,results)
        
        
        cv2.imshow("frame" , image)
       # cv2.imwrite("image.jpg", image)
        if cv2.waitKey(1)==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [7]:
data_path = os.path.join("mpn_data/")
actions= np.array(['Thankyou', "Hello","To_save","Help","Good","Hope","You"])#"A","B"])#"C","G"])
#actions= np.array(["D"])


no_sequences = 60 #30 Videos for each actions
sequence_length = 30 ##30 Frames


In [8]:
#Making Folders 

for action in actions:
    for s in range(no_sequences):
        try:
            os.makedirs(os.path.join(data_path, action, str(s)))
        except:
            pass


In [9]:
##collecting images

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for s in range(no_sequences):
            for i in range(sequence_length):
                ret, frame = cap.read()
                #Using Mediapipe

                image, results = mediapipe_detection(frame, holistic)   

                #Drawing Ladmarks
                draw_landmarks2(image,results)

                if i==0:
                    cv2.putText(image, "Starting", (120,200), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image, "Collecting Fraes for  {} Video Number {}".format(action,s ), (15,12), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.waitKey(1000)
                else:
                    cv2.putText(image, "Collecting Frames for  {} Video Number {}".format(action,s), (15,12), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(data_path, action,str(s),str(i))
                np.save(npy_path,keypoints)
                cv2.imshow("frame" , image)

                # cv2.imwrite("image.jpg", image)
                if cv2.waitKey(1)==ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map = {label:num for num,label in enumerate(actions)}

label_map

{'Thankyou': 0,
 'Hello': 1,
 'To_save': 2,
 'Help': 3,
 'Good': 4,
 'Hope': 5,
 'You': 6}

In [10]:
sequences, labels = [],[]
for action in actions:
    for seq in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(data_path,action , str(seq),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        
np.array(sequences).shape

(420, 30, 1662)

In [11]:
y = to_categorical(labels).astype(int)
X = np.array(sequences)
X.shape

(420, 30, 1662)

In [12]:
##SPlitting Data

X_train , X_test, y_train, y_test = train_test_split(X,y,test_size = 0.05)



In [13]:
# y_train.shape

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [22]:
log_dir = os.path.join("logs")
tb_callback = TensorBoard(log_dir=log_dir)

In [23]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer= "adam", loss = "categorical_crossentropy",metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs = 150,callbacks= [tb_callback])

Train on 399 samples
Epoch 1/150
399/399 [==============================] - 4s 11ms/sample - loss: 2.4374 - categorical_accuracy: 0.1454
Epoch 2/150
399/399 [==============================] - 2s 6ms/sample - loss: 1.9664 - categorical_accuracy: 0.1479
Epoch 3/150
399/399 [==============================] - 2s 6ms/sample - loss: 1.9496 - categorical_accuracy: 0.1679
Epoch 4/150
399/399 [==============================] - 2s 6ms/sample - loss: 1.9201 - categorical_accuracy: 0.1980
Epoch 5/150
399/399 [==============================] - 2s 6ms/sample - loss: 1.8425 - categorical_accuracy: 0.2256
Epoch 6/150
399/399 [==============================] - 2s 6ms/sample - loss: 1.6808 - categorical_accuracy: 0.2732
Epoch 7/150
399/399 [==============================] - 2s 6ms/sample - loss: 1.6735 - categorical_accuracy: 0.24310s - loss: 1.6782 - categorical_accuracy: 0.24
Epoch 8/150
399/399 [==============================] - 2s 6ms/sample - loss: 1.4355 - categorical_accuracy: 0.3609
Epoch 9/150


399/399 [==============================] - 2s 6ms/sample - loss: 0.6436 - categorical_accuracy: 0.7218
Epoch 71/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.5357 - categorical_accuracy: 0.7669
Epoch 72/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.8344 - categorical_accuracy: 0.6441
Epoch 73/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.6861 - categorical_accuracy: 0.6892
Epoch 74/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.5887 - categorical_accuracy: 0.7644
Epoch 75/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.4986 - categorical_accuracy: 0.7719
Epoch 76/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.4838 - categorical_accuracy: 0.7970
Epoch 77/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.6249 - categorical_accuracy: 0.7343
Epoch 78/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.5038 

399/399 [==============================] - 2s 6ms/sample - loss: 0.0424 - categorical_accuracy: 0.9850
Epoch 140/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.0320 - categorical_accuracy: 0.9950
Epoch 141/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.0266 - categorical_accuracy: 0.9925
Epoch 142/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.0298 - categorical_accuracy: 0.9850
Epoch 143/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.0530 - categorical_accuracy: 0.97741s - loss: 0.0403 - categorical_ac
Epoch 144/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.1164 - categorical_accuracy: 0.9499
Epoch 145/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.1830 - categorical_accuracy: 0.9474
Epoch 146/150
399/399 [==============================] - 2s 6ms/sample - loss: 0.1031 - categorical_accuracy: 0.9599
Epoch 147/150
399/399 [=====================

In [60]:

res = model.predict(X_test)

print(res)
model.summary()

[[2.08563972e-04 5.92446577e-06 1.67406013e-03 5.45379237e-07
  1.69252267e-03 6.28904527e-05 9.90751266e-01 2.21399910e-06
  1.32138899e-03 2.68479506e-03 1.29575748e-03 2.99972337e-04]
 [2.72457284e-30 0.00000000e+00 1.74872153e-20 4.18375974e-36
  9.69439548e-25 4.19537983e-28 3.21127190e-24 0.00000000e+00
  4.79875267e-29 6.92112645e-10 4.08500707e-12 1.00000000e+00]
 [1.28851781e-04 2.03366473e-01 3.25196562e-03 3.46894958e-05
  8.64347618e-04 4.32914276e-05 7.59682208e-02 2.91156888e-01
  3.27217460e-01 9.01215374e-02 4.59866365e-03 3.24762193e-03]
 [1.38595671e-04 2.00853303e-01 1.24173649e-02 8.63568202e-05
  1.56901814e-02 9.10296294e-05 3.50042023e-02 4.81850617e-02
  2.71753758e-01 3.44852805e-01 6.00810014e-02 1.08463401e-02]
 [2.39869922e-01 1.37397992e-05 2.28760272e-01 2.78921336e-01
  9.44785625e-02 1.02557480e-01 1.71606299e-02 9.03174150e-05
  6.19769562e-05 5.79750980e-04 2.39098649e-02 1.35961166e-02]
 [4.62669939e-01 6.18245255e-12 2.12482666e-03 2.92237801e-05
  5

In [61]:
print(actions[np.argmax(res[1])])

print(actions[np.argmax(y_test[1])])

G
G


In [24]:
model.save("common_words_1.h5")

In [25]:
##MODEL EVALUATION

from sklearn.metrics import multilabel_confusion_matrix,accuracy_score
yhat = model.predict(X_test)

ytrue= np.argmax(y_test,axis =1 ).tolist()
yhat = np.argmax(yhat,axis = 1 ).tolist()

multilabel_confusion_matrix(ytrue, yhat)


array([[[17,  2],
        [ 1,  1]],

       [[16,  0],
        [ 1,  4]],

       [[17,  1],
        [ 0,  3]],

       [[18,  0],
        [ 0,  3]],

       [[18,  0],
        [ 0,  3]],

       [[17,  0],
        [ 2,  2]],

       [[19,  1],
        [ 0,  1]]], dtype=int64)

In [26]:
accuracy_score(ytrue , yhat)

0.8095238095238095

In [16]:
import tensorflow as tf
model = tf.keras.models.load_model("label_8_3.h5")

In [28]:
##Final testing

import tensorflow as tf
sequences = []#concatenating 30 frames
sentences = []
predictions = []
threshold = 0.45

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        #Using Mediapipe

        image, results = mediapipe_detection(frame, holistic)   
        
        #Drawing Ladmarks
        draw_landmarks2(image,results)
        
        ##making predictions
        
        keypoints = extract_keypoints(results)
        #sequences.insert(0,keypoints)
        sequences.append(keypoints)
        sequences = sequences[-30:]
        
        if len(sequences)==30:
            res = model.predict(np.expand_dims(sequences,axis = 0))[0]
            predictions.append(np.argmax(res))
            
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)]>threshold:
                    
                    if len(sentences)>0:
                        
                        if actions[np.argmax(res)]!=sentences[-1]:
                            print(actions[np.argmax(res)])
                            sentences.append(actions[np.argmax(res)])
                            
                    else:
                        sentences.append(actions[np.argmax(res)])
            if len(sentences)>5:
                sentences = sentences[-5:]
    
        cv2.rectangle(image, (0,0),(640,40), (245,117,16),-1 )
        cv2.putText(image,"".join(sentences),(3,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        
        cv2.imshow("frame" , image)
       # cv2.imwrite("image.jpg", image)
        if cv2.waitKey(1)==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

Good
Hello
You
Good
To_save
Thankyou
Hello
Thankyou
To_save
Thankyou
Help
Thankyou
You


In [ ]:
model1 = tf.model.